In [ ]:
"""
Replica in Python:
Bachelor's Thesis: Teacher Density and Student Achievement
Regression for Table 1 - 3 in thesis

"""

import pandas as pd
df = pd.read_csv("paneldata_finale.csv")

# ---------------------------
# Convert key columns to numeric
# ---------------------------
num_cols = [
    "Ar",
    "merit_snitt",
    "Elever_per_larare",
    "Andel_med_lararlegitimation_och_",
    "Andel_svenskar_procent",
    "Foraldrar_utbildning_procent",
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["Skol_enhetskod", "Ar"]).copy()
df["Ar"] = df["Ar"].astype(int)

# ---------------------------
# School-specific linear trend
# ---------------------------
df = df.sort_values(["Skol_enhetskod", "Ar"])
df["trend"] = df.groupby("Skol_enhetskod").cumcount() + 1

# ---------------------------
# Panel index + year dummies
# ---------------------------
df = df.set_index(["Skol_enhetskod", "Ar"]).sort_index()

year_dummies = pd.get_dummies(
    df.reset_index()["Ar"].astype(int),
    prefix="Ar",
    drop_first=True
)
year_dummies.index = df.index
df = pd.concat([df, year_dummies], axis=1)
year_cols = list(year_dummies.columns)

# ---------------------------
# Six models per sample
# ---------------------------
def run_models(dsub):
    # Model 1: RE + year FE
    cols1 = ["merit_snitt", "Elever_per_larare", "Huvudman"] + year_cols
    d1 = dsub[cols1].dropna().copy()
    y1 = d1["merit_snitt"]
    X1 = sm.add_constant(d1[["Elever_per_larare"] + year_cols], has_constant="add")
    m1 = RandomEffects(y1, X1).fit(cov_type="clustered", clusters=d1["Huvudman"])

    # Model 2: RE + year FE
    cols2 = ["merit_snitt", "Elever_per_larare", "Andel_med_lararlegitimation_och_", "Huvudman"] + year_cols
    d2 = dsub[cols2].dropna().copy()
    y2 = d2["merit_snitt"]
    X2 = sm.add_constant(d2[["Elever_per_larare", "Andel_med_lararlegitimation_och_"] + year_cols], has_constant="add")
    m2 = RandomEffects(y2, X2).fit(cov_type="clustered", clusters=d2["Huvudman"])

    # Model 3: RE + year FE
    cols3 = ["merit_snitt", "Elever_per_larare", "Andel_med_lararlegitimation_och_", "Andel_svenskar_procent", "Huvudman"] + year_cols
    d3 = dsub[cols3].dropna().copy()
    y3 = d3["merit_snitt"]
    X3 = sm.add_constant(d3[["Elever_per_larare", "Andel_med_lararlegitimation_och_", "Andel_svenskar_procent"] + year_cols], has_constant="add")
    m3 = RandomEffects(y3, X3).fit(cov_type="clustered", clusters=d3["Huvudman"])

    # Model 4: RE + year FE
    cols4 = ["merit_snitt", "Elever_per_larare", "Andel_med_lararlegitimation_och_", "Andel_svenskar_procent", "Foraldrar_utbildning_procent", "Huvudman"] + year_cols
    d4 = dsub[cols4].dropna().copy()
    y4 = d4["merit_snitt"]
    X4 = sm.add_constant(d4[["Elever_per_larare", "Andel_med_lararlegitimation_och_", "Andel_svenskar_procent", "Foraldrar_utbildning_procent"] + year_cols], has_constant="add")
    m4 = RandomEffects(y4, X4).fit(cov_type="clustered", clusters=d4["Huvudman"])

    # Model 5: School FE + year FE
    cols5 = ["merit_snitt", "Elever_per_larare", "Andel_med_lararlegitimation_och_", "Andel_svenskar_procent", "Foraldrar_utbildning_procent", "Huvudman"]
    d5 = dsub[cols5].dropna().copy()
    y5 = d5["merit_snitt"]
    X5 = d5[["Elever_per_larare", "Andel_med_lararlegitimation_och_", "Andel_svenskar_procent", "Foraldrar_utbildning_procent"]]
    m5 = PanelOLS(y5, X5, entity_effects=True, time_effects=True).fit(
        cov_type="clustered", clusters=d5["Huvudman"]
    )

    # Model 6: School FE + year FE + School-specific linear trends
    d6 = dsub.reset_index()[[
        "Skol_enhetskod", "Ar", "Huvudman", "trend", "merit_snitt",
        "Elever_per_larare", "Andel_med_lararlegitimation_och_",
        "Andel_svenskar_procent", "Foraldrar_utbildning_procent"
    ]].dropna().copy()

    f6 = (
        "merit_snitt ~ Elever_per_larare + Andel_med_lararlegitimation_och_ + "
        "Andel_svenskar_procent + Foraldrar_utbildning_procent + "
        "C(Ar) + C(Skol_enhetskod) + C(Skol_enhetskod):trend"
    )
    m6 = smf.ols(f6, data=d6).fit(
        cov_type="cluster",
        cov_kwds={"groups": d6["Huvudman"]}
    )

    return m1, m2, m3, m4, m5, m6

samples = {
    "Panel A. All schools": df,
    "Panel B. Voucher schools": df[df["Typ_av_huvudman"] == "Enskild"],
    "Panel C. Municipal schools": df[df["Typ_av_huvudman"] == "Kommunal"],
}
results = {name: run_models(data) for name, data in samples.items()}

# ---------------------------
# Construct table:
# ---------------------------
show_vars = [
    "Elever_per_larare",
    "Andel_med_lararlegitimation_och_",
    "Andel_svenskar_procent",
    "Foraldrar_utbildning_procent",
]
model_names = ["M1", "M2", "M3", "M4", "M5", "M6"]
fe_flag     = ["No", "No", "No", "No", "Yes", "Yes"]
yearfe_flag = ["Yes", "Yes", "Yes", "Yes", "Yes", "Yes"]
trend_flag  = ["No", "No", "No", "No", "No", "Yes"]

def stars(p):
    if pd.isna(p):
        return ""
    if p < 0.01:
        return "***"
    if p < 0.05:
        return "**"
    if p < 0.10:
        return "*"
    return ""

def coef_se_cell(model, var, is_statsmodels=False):
    if is_statsmodels:
        params, ses, pvals = model.params, model.bse, model.pvalues
    else:
        params, ses, pvals = model.params, model.std_errors, model.pvalues

    if var not in params.index:
        return ""
    return f"{params[var]:.3f}{stars(pvals[var])}\n({ses[var]:.3f})"

def build_panel_table(models):
    idx = show_vars + ["Observations", "Fixed Effects", "Year Fixed Effects", "Linear Trend"]
    out = pd.DataFrame(index=idx, columns=model_names)

    for j, m in enumerate(models):
        is_sm = (j == 5)  # Model 6 is statsmodels OLS
        for v in show_vars:
            out.loc[v, model_names[j]] = coef_se_cell(m, v, is_statsmodels=is_sm)

        out.loc["Observations", model_names[j]] = f"{int(m.nobs):,}"
        out.loc["Fixed Effects", model_names[j]] = fe_flag[j]
        out.loc["Year Fixed Effects", model_names[j]] = yearfe_flag[j]
        out.loc["Linear Trend", model_names[j]] = trend_flag[j]

    return out

tables = {}
for panel_name, models in results.items():
    tbl = build_panel_table(models)
    tables[panel_name] = tbl

    print("\n" + "=" * 90)
    print(panel_name)
    print("=" * 90)
    print(tbl.fillna(""))



Panel A. All schools
                                                 M1                 M2  \
Elever_per_larare                 0.952***\n(0.183)  0.987***\n(0.181)   
Andel_med_lararlegitimation_och_                     0.082***\n(0.026)   
Andel_svenskar_procent                                                   
Foraldrar_utbildning_procent                                             
Observations                                 11,049             11,049   
Fixed Effects                                    No                 No   
Year Fixed Effects                              Yes                Yes   
Linear Trend                                     No                 No   

                                                 M3                 M4  \
Elever_per_larare                 0.861***\n(0.181)  0.758***\n(0.157)   
Andel_med_lararlegitimation_och_   0.052**\n(0.025)     0.003\n(0.023)   
Andel_svenskar_procent            0.476***\n(0.034)  0.262***\n(0.029)   
Foraldrar_utbil